In [3]:
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

random_state = 0
np.random.seed(0)

In [4]:
df_gender = pd.read_csv('gender_submission.csv')
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


# Data cleansing

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
df_train['Age'].fillna(value = df_train['Age'].mean(), inplace = True)
df_train['Age'] = df_train['Age'].astype(int)

df_test['Age'].fillna(value = df_test['Age'].mean(), inplace = True)
df_test['Age'] = df_test['Age'].astype(int)

df_train['Embarked'].fillna(value = df_train['Embarked'].mode, inplace = True)
df_test['Embarked'].fillna(value = df_test['Embarked'].mode, inplace = True)

df_test['Fare'].fillna(value = df_test['Fare'].mean(), inplace = True)

def sexo_binario(sexo):
  if sexo == 'male':
    return 0
  else:
    return 1

df_train['Sex'] = df_train['Sex'].map(sexo_binario)
df_test['Sex'] = df_test['Sex'].map(sexo_binario)

df_train['Alone'] = (df_train['SibSp'] + df_train['Parch'] > 0).astype(int)
df_train.drop(columns=['Parch', 'Alone'], inplace=True)
df_train.rename(columns={'SibSp': 'Accompanied'}, inplace=True)

df_test['Alone'] = (df_test['SibSp'] + df_test['Parch'] > 0).astype(int)
df_test.drop(columns=['Parch', 'Alone'], inplace=True)
df_test.rename(columns={'SibSp': 'Accompanied'}, inplace=True)

In [7]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    int64  
 5   Age          891 non-null    int32  
 6   Accompanied  891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Fare         891 non-null    float64
 9   Cabin        204 non-null    object 
 10  Embarked     891 non-null    object 
dtypes: float64(1), int32(1), int64(5), object(4)
memory usage: 73.2+ KB


In [8]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    int64  
 4   Age          418 non-null    int32  
 5   Accompanied  418 non-null    int64  
 6   Ticket       418 non-null    object 
 7   Fare         418 non-null    float64
 8   Cabin        91 non-null     object 
 9   Embarked     418 non-null    object 
dtypes: float64(1), int32(1), int64(4), object(4)
memory usage: 31.2+ KB


# First model - Sex and age

In [9]:
X_1_train = df_train[['Sex', 'Age']]
X_1_test = df_test[['Sex', 'Age']]

y_train = df_train['Survived']

first_model = RandomForestClassifier(n_estimators = 100, n_jobs =- 1, random_state = random_state)
first_model.fit(X_1_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=0)

In [10]:
first_result = first_model.predict(X_1_test)
first_result = pd.Series(first_result, index = df_test['PassengerId'], name = 'Survived')
first_result.to_csv('first_result.csv', header = True)

# Second model - Sex, age and accompanied

In [11]:
X_2_train = df_train[['Sex', 'Age', 'Accompanied']]
X_2_test = df_test[['Sex', 'Age', 'Accompanied']]

second_model = RandomForestClassifier(n_estimators = 100, n_jobs =- 1, random_state = random_state)
second_model.fit(X_2_train, y_train)

RandomForestClassifier(n_jobs=-1, random_state=0)

In [12]:
second_result = second_model.predict(X_2_test)
second_result = pd.Series(second_result, index = df_test['PassengerId'], name = 'Survived')
second_result.to_csv('first_result.csv', header = True)

# Third model

In [13]:
x_3_train, x_3_test, y_3_train, y_3_test = train_test_split(X_1_train, y_train, test_size = 0.5)

third_model = RandomForestClassifier(n_estimators = 100, n_jobs=-1, random_state = random_state)
third_model.fit(x_3_train, y_3_train)

RandomForestClassifier(n_jobs=-1, random_state=0)

In [14]:
third_result = third_model.predict(x_3_test)
np.mean(third_result == y_3_test)

0.7690582959641256

# Base value - Using only the gender as parameter

In [15]:
base_predict = (X_1_train['Sex'] == 1).astype(np.int64)
np.mean(base_predict == y_train)

0.7867564534231201

# Cross Validation

In [23]:
X = df_train[['Sex', 'Age']]
y = df_train['Survived']

In [28]:
kf = KFold(5, shuffle = True, random_state = random_state)
results = []

for train_lines, valid_lines in kf.split(X):
    
    print("Train:", train_lines.shape[0])
    print("Test:", valid_lines.shape[0])
    
    X_train, X_valid = X.iloc[train_lines], X.iloc[valid_lines]
    y_train, y_valid = y.iloc[train_lines], y.iloc[valid_lines]
    
    model = RandomForestClassifier(n_estimators = 100, n_jobs =-1, random_state = random_state)
    model.fit(X_train, y_train)
    prediction = model.predict(X_valid)
    
    accuracy = np.mean(y_valid == prediction)
    results.append(accuracy)
    
    print("Accuracy:", accuracy)
    print()

Train: 712
Test: 179
Accuracy: 0.7988826815642458

Train: 713
Test: 178
Accuracy: 0.7303370786516854

Train: 713
Test: 178
Accuracy: 0.7752808988764045

Train: 713
Test: 178
Accuracy: 0.797752808988764

Train: 713
Test: 178
Accuracy: 0.7865168539325843



In [29]:
np.mean(results)

0.7777540644027369